In [41]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [46]:
def read_dataframe(filename, verbose = False):
    df = pd.read_parquet(filename)

    #Q1
    if verbose:
        print(len(df.columns), 'columns')

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.apply(lambda td: td.total_seconds() / 60)

    #Q2
    if verbose:
        print('Standard deviation of the trips duration', round(df.duration.std(), 2))

    #Q3
    df_full_len = len(df)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    if verbose:
        print(f"{round(len(df) / df_full_len * 100)}% of the records left")
    
    categorical = ['PULocationID', 'DOLocationID']
    
    df[categorical] = df[categorical].astype(str)

    return df


In [47]:
df_train = read_dataframe('./data/yellow_tripdata_2023-01.parquet', True)

19 columns
Standard deviation of the trips duration 42.59
98% of the records left


In [49]:
categorical = ['PULocationID', 'DOLocationID']

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

In [35]:
#Q4
f'Number of columns {X_train.get_shape()[1]}'

'Number of columns 515'

In [37]:
target = 'duration'
y_train = df_train[target].values

In [45]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

#Q5
f'RMSE on train {round(mean_squared_error(y_train, y_pred, squared=False), 2)}'

'RMSE on train 7.65'

In [51]:
df_val = read_dataframe('./data/yellow_tripdata_2023-02.parquet')

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

y_val = df_val[target].values

y_pred = lr.predict(X_val)

#Q5
f'RMSE on validation {round(mean_squared_error(y_val, y_pred, squared=False), 2)}'

'RMSE on validation 7.81'